In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('train.csv', index_col=0)

data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
y = data.SalePrice

X = data.drop(['SalePrice'], axis=1)

In [4]:
X.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [5]:
numerical_cols = [col for col in X.columns if data[col].dtype in ['int64', 'float64']]

categorical_cols = [col for col in X.columns if data[col].dtype == 'object' and data[col].nunique() < 10 and data[col].isnull().sum() == 0]

In [6]:
X[numerical_cols].head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


In [7]:
X[categorical_cols].head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
2,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,1Fam,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
5,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,1Fam,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [8]:
# Selected columns
my_cols = numerical_cols + categorical_cols

X = X[my_cols]

X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=22)

In [10]:
imputer = SimpleImputer(strategy='constant')

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [11]:
X_train.isnull().sum().head(10)

MSSubClass        0
LotFrontage     212
LotArea           0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
MasVnrArea        6
BsmtFinSF1        0
BsmtFinSF2        0
dtype: int64

In [12]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1071,20,72.0,10152,5,5,1956,1956,120.0,586,0,...,TA,CBlock,GasA,TA,Y,TA,Typ,Y,WD,Normal
1380,80,73.0,9735,5,5,2006,2007,0.0,0,0,...,TA,PConc,GasA,Gd,Y,Gd,Typ,Y,WD,Normal
139,60,73.0,9066,8,5,1999,2000,320.0,668,0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
668,20,65.0,8125,6,5,1994,1998,258.0,1138,0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1216,20,99.0,7094,5,5,1966,1966,0.0,180,374,...,TA,CBlock,GasA,TA,Y,TA,Typ,Y,WD,Normal


In [13]:
imp_X_trn = pd.DataFrame(imputer.fit_transform(X_train[numerical_cols]))
imp_X_val = pd.DataFrame(imputer.transform(X_train[numerical_cols]))

In [14]:
imp_X_trn.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,20.0,72.0,10152.0,5.0,5.0,1956.0,1956.0,120.0,586.0,0.0,...,286.0,0.0,20.0,0.0,0.0,192.0,0.0,0.0,6.0,2007.0
1,80.0,73.0,9735.0,5.0,5.0,2006.0,2007.0,0.0,0.0,0.0,...,400.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2008.0
2,60.0,73.0,9066.0,8.0,5.0,1999.0,2000.0,320.0,668.0,0.0,...,660.0,224.0,106.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0
3,20.0,65.0,8125.0,6.0,5.0,1994.0,1998.0,258.0,1138.0,0.0,...,575.0,224.0,42.0,0.0,0.0,0.0,0.0,0.0,10.0,2008.0
4,20.0,99.0,7094.0,5.0,5.0,1966.0,1966.0,0.0,180.0,374.0,...,384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0


In [15]:
imp_X_trn.columns = X_train[numerical_cols].columns
imp_X_val.columns = X_valid[numerical_cols].columns

imp_X_trn.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20.0,72.0,10152.0,5.0,5.0,1956.0,1956.0,120.0,586.0,0.0,...,286.0,0.0,20.0,0.0,0.0,192.0,0.0,0.0,6.0,2007.0
1,80.0,73.0,9735.0,5.0,5.0,2006.0,2007.0,0.0,0.0,0.0,...,400.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2008.0
2,60.0,73.0,9066.0,8.0,5.0,1999.0,2000.0,320.0,668.0,0.0,...,660.0,224.0,106.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0
3,20.0,65.0,8125.0,6.0,5.0,1994.0,1998.0,258.0,1138.0,0.0,...,575.0,224.0,42.0,0.0,0.0,0.0,0.0,0.0,10.0,2008.0
4,20.0,99.0,7094.0,5.0,5.0,1966.0,1966.0,0.0,180.0,374.0,...,384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0


In [17]:
imp_X_trn.index = X_train[numerical_cols].index

imp_X_trn.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1071,20.0,72.0,10152.0,5.0,5.0,1956.0,1956.0,120.0,586.0,0.0,...,286.0,0.0,20.0,0.0,0.0,192.0,0.0,0.0,6.0,2007.0
1380,80.0,73.0,9735.0,5.0,5.0,2006.0,2007.0,0.0,0.0,0.0,...,400.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2008.0
139,60.0,73.0,9066.0,8.0,5.0,1999.0,2000.0,320.0,668.0,0.0,...,660.0,224.0,106.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0
668,20.0,65.0,8125.0,6.0,5.0,1994.0,1998.0,258.0,1138.0,0.0,...,575.0,224.0,42.0,0.0,0.0,0.0,0.0,0.0,10.0,2008.0
1216,20.0,99.0,7094.0,5.0,5.0,1966.0,1966.0,0.0,180.0,374.0,...,384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0


In [16]:
enc_cat_trn = pd.DataFrame(encoder.fit_transform(X_train[categorical_cols]))
enc_cat_val = pd.DataFrame(encoder.transform(X_train[categorical_cols]))